# Calculate CS shiftx2

In [1]:
import MDAnalysis as mda
import sys
import shutil
import pandas as pd
sys.path.append("..")
from src.features.build_features import get_chemical_shifts
import os
import numpy as np
# from spc_imports import *
# set_up_plt()

/home/sperez/data_partition/bin2/anaconda3/envs/nmr_assign_state_for_shiftx2/lib/python3.5/site-packages/MDAnalysis/due.py:88: UserWarning: No module named 'mock'
  warnings.warn(str(err))


In [2]:
raw_data_dir = '../data/raw/'
interim_data_dir = '../data/interim/'
processed_data_dir = '../data/processed/'
external_data_dir = '../data/external/'

In [3]:
states = { 
#               '5VKH_lb': {'begin': 0,
#                       'end': 1.e+20},
    '3FB5_lb' : {'begin': 400000.,
                      'end': 1000000.},
#           '5VK6_lb': {'begin': 0.,
#                       'end': 350000.},
#           '5VKE_lb': {'begin': 0,
#                       'end': 1000000.}
         }

## How to run this Notebook

This notebook must be used after processing the trajectories with `calculate_cs.ipynb`

This notebook must be run with the environment created with: `environment_for_shiftx2.yml`. This is because shiftx2 is hard to install and use otherwise.

Shiftx2 was written in python2. The ominia channel version of shiftx2 for python3.5 is essentially only a wrapper calling shiftx2 with python2 from python3.5.
In order to not have any problem of python3 being picked up instead of python2, it is best to:
```bash
cd /path/to/anaconda3/envs/nmr_assign_state_for_shiftx2/share/shiftx2
find . -iname "[a-zA-Z]*.py" -exec sed -i '1s/python/python2/' {} \;
find . -iname "[a-zA-Z]*.py" -exec sed -i 's/"python/"python2/' {} \;
```





### Get Chemical Shifts

In [28]:
def get_chemical_shifts(univ,temp_dir='./',split_size=100,skip=1,method='sparta_plus',temperature=298.,pH=5.0, protein_selection = 'protein'):
    import MDAnalysis as mda
    import mdtraj as md
    import numpy as np
    import pandas as pd
    import os
    from tqdm import tqdm
    import warnings
    warnings.filterwarnings("ignore")

    '''
    Use sparta_plus with the interface of mdtraj to get the chemical shifts of the trajectory. The segids are concatenated as independent trajectories.
    The protein is assumed to have 4 identical subunits. 

    Parameters
    ----------
    univ : Universe object that contains the protein.
    temp_dir : Temporary directory for the tmp trajectory.
    split_size : How to split the trajectory so the /tmp does not fill.
    skip : Amount of frames skiped.
    pH: pH for shiftx2 calculations.
    temperature: temperature for shiftx2 calculations.
    protein_selection: protein selection for MDAnalysis.
      
    Returns
    -------
    df: Dataframe of chemical shifts (n_cs_nuclei,n_frames*n_segids)
    '''
    assert isinstance(temp_dir,str), {temp_dir} + ' should be a str'
    assert isinstance(protein_selection,str), 'protein_selection should be a str'

    assert isinstance(split_size,int), split_size +  ' should be a int'
    assert isinstance(skip,int), skip + ' should be a int'
    assert isinstance(temperature,float), str(temperature) + ' should be a float'
    assert isinstance(pH,float), str(pH) + ' should be a float'
    
    sel_protein=univ.select_atoms(protein_selection)
    resids = sel_protein.residues.resids.copy()
    gap = 10
    last_resid = -gap
    for i,seg in enumerate(univ.segments):
        seg.residues.resids = seg.residues.resids + gap + last_resid
        last_resid = seg.residues.resids[-1]
    renumber = sel_protein.residues.resids
    resids_dic = { i:j  for i,j in zip(renumber,resids)}
    sel_protein.segments.segids = 'A'
    
    xtc = temp_dir+"/tmp.xtc"
    pdb = temp_dir+"/tmp.pdb"
    
    
    #Which method:
    if method == 'sparta_plus': 
        method_function = md.nmr.chemical_shifts_spartaplus
    elif method == 'ppm': 
        method_function = md.nmr.chemical_shifts_ppm
    elif method == 'shiftx2': 
        method_function = md.nmr.chemical_shifts_shiftx2
    else:
        raise Exception('Method' + method + 'not recognized.')
    
    
     
    with mda.Writer(xtc, n_atoms=sel_protein.atoms.n_atoms) as W:
        for ts in univ.trajectory[::skip]:
            W.write(sel_protein)
    sel_protein.write(pdb)

    #Load data and calculate NMR-CS
    trj= md.load(xtc,top=pdb)
    n_frames=trj.n_frames
    list_of_splits = []
    for j in tqdm(range(0,n_frames,split_size),
                  leave=True,smoothing=1):
        if method == 'shiftx2':
            list_of_splits.append(
           method_function(trj[j:j+split_size],pH=pH,temperature=temperature)
            )
        else:
            list_of_splits.append(
           method_function(trj[j:j+split_size])
            )
            
    df = pd.concat(list_of_splits,axis=1)
    index = pd.MultiIndex.from_arrays([[ resids_dic[i[0]] for i in df.index ], [ i[1] for i in df.index ]], names = ['resid', 'nuclei'])
    df.index = index
    df = df.T
    df = df.loc[:,pd.IndexSlice[:,['N', 'CA', 'CB', 'C']]]
    dictionary = {}
    for col in np.unique(df.columns):
        k = df.loc[:,col]
        if k.shape[1] == 4:
            dictionary[col] = np.array(k).reshape(k.shape[0]*k.shape[1])
    df = pd.DataFrame(dictionary)
    df = df.rename_axis(['resid', 'nuclei'], axis=1)
    os.remove(xtc)
    os.remove(pdb)
    return df

In [29]:
for method in ['shiftx2']:
    for state in states.keys():
        univ = mda.Universe(interim_data_dir + state+ '/protein_sk1_pbc.pdb',
                            interim_data_dir + state+'/protein_sk1_pbc.xtc')
        df = get_chemical_shifts(univ, '../data/processed/',method=method,pH=4.,skip=1, protein_selection='protein and not resid 26 121')
        df.to_pickle(processed_data_dir+state+'/CS_'+method+'_'+state+'.pkl')

TypeError: Selection type must be compatible with slicing the coordinates